Attention Mechanism
===================
   - seq2seq 모델의 문제점을 개선
      1. 하나의 고정 길이 벡터에 모든 정보를 압축해 정보 손실 발생
      2. RNN의 문제점인 기울기 소실(Gradient vanising)이 동일하게 발생

   - seq2seq 문제를 개선하기 위해 Attention Machanism이 탄생
       - Attention Mechanism은 디코더가 예측하는 시점마다 인코더의 전체 입력 문장을 다시 한번 참조
       - 이때 전체 입력 문장을 단순히 참조하지 않고, 예측할 단어와 연관이 있는 단어를 집중(Attention)해서 참조

__Attention Mechanism 종류__
   - attention mechanism에는 스코어 계산 방식의 차이에 따라 다양한 종류가 존재
   - dot, scaled dot, general, concat, kocation-base
   - 용어 정리
      1. s : querys(t 시점에서의 디코더 셀의 은닉상태)
      2. h : keys(모든 시점의 인코더 셀 은닉상태)
      3. W : 학습 가능한 가중치 행렬

__Attention Mechanism 과정__
   - Attention Mechanism 중 가장 기초적인 dot-product attention을 예제로 적용 가정을 학습
   - Attention Mechanism 과정
      1. Attention score 계산
      2. softmax 함수를 통한 attention distribution 계산
      3. 각 인코더의 어텐션 가중치와 은닉 상태를 가중합하여 어텐션 값 계산
      4. 어텐션 값과 디코더의 t 시점의 은닉 상태를 연결
      5. 출력층 연산의 입력이 되는 s 계산

__데이터 전처리__

In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Embedding, LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import urllib3
import zipfile
import numpy as np
import pandas as pd

In [2]:
# pandas 형식으로 불러오기
lines = pd.read_csv("./nlp_data/fra.txt", names=['src', 'tar', 'lic'], sep='\t')
del lines['lic']

lines = lines.loc[:, 'src':'tar']
lines = lines[0:60000]
lines.tar = lines.tar.apply(lambda x: '\t' + x + '\n')

src_vocab = set()
for line in lines.src:
    for char in line:
        src_vocab.add(char)

tar_vocab = set()
for line in lines.tar:
    for char in line:
        tar_vocab.add(char)
        
src_vocab = sorted(list(src_vocab))
tar_vocab = sorted(list(tar_vocab))

src_vocab_size = len(src_vocab) + 1
tar_vocab_size = len(tar_vocab) + 1

src_to_idx = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_idx = dict([(word, i+1) for i, word in enumerate(tar_vocab)])

In [3]:
# Encoder input

encoder_input = []
for line in lines.src:
    encoder_input.append([src_to_idx[w] for w in line])
    
# Decoder input

decoder_input = []
for line in lines.tar:
    decoder_input.append([tar_to_idx[w] for w in line])
    
# Decoder target

decoder_target = []
for line in lines.tar:
    decoder_target.append([tar_to_idx[w] for w in line if w != '\t'])
    
# padding
# 문장 최대길이
max_src_len = max([len(line) for line in lines.src])
max_tar_len = max([len(line) for line in lines.tar])

encoder_input = pad_sequences(encoder_input, maxlen=max_src_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_tar_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')

# one-hot encoding
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)


__Attention Mechanism 모델__

인코더(Encoder)

In [4]:
encoder_inputs = Input(shape=(None, src_vocab_size))
encoder_lstm = LSTM(256, return_state=True)

encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

디코터(Decoder)
   - 디코더에서는 seq2seq와는 다르게 attention layer를 추가함
   - S_는 은닉 상태와 디코더의 최종 출력을 연결한 결과, 연결할 때 형상을 맞춰주기 위해 축을 추가함
   - attention layer는 디코더의 은닉 상태와 인코더 은닉 상태 전체를 받아 컨텍스트 벡터를 생성함
   - 이 때 attention layer는 앞서 설명한 과정 중 1~3번째를 수행, 나머지는 사용자가 연결해주어야 함
   - 마지막으로 생성한 컨텍스트 벡터와 디코더의 은닉 상태 전체를 이어 softmax layer에 투입, 인덱스를 예측함

In [5]:
import tensorflow as tf
from keras.layers import Attention


decoder_inputs = Input(shape=(None, tar_vocab_size))
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

S_ = tf.concat([state_h[:, tf.newaxis, :], decoder_outputs[:, :-1, :]], axis=1)

attention = Attention()
context_vector = attention([S_, encoder_outputs])
concat = tf.concat([decoder_outputs, context_vector], axis=-1)
decoder_softmax_layer = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(concat)

모델 구성 및 학습
   - 구성하는 방법은 seq2seq와 동일함
   - attention mechanism을 활용해 학습 시간이 절반 가량 준 것을 확인할 수 있음

In [6]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [7]:
model.fit(x=[encoder_input, decoder_input], y=decoder_target,
          batch_size=128,
          epochs=25,
          validation_split=0.2)

Epoch 1/25
375/375 [==============================] - 14s 28ms/step - loss: 0.8960 - val_loss: 0.7975
Epoch 2/25
375/375 [==============================] - 9s 24ms/step - loss: 0.5515 - val_loss: 0.6452
Epoch 3/25
375/375 [==============================] - 9s 25ms/step - loss: 0.4561 - val_loss: 0.5706
Epoch 4/25
375/375 [==============================] - 9s 24ms/step - loss: 0.4039 - val_loss: 0.5259
Epoch 5/25
375/375 [==============================] - 9s 25ms/step - loss: 0.3673 - val_loss: 0.4988
Epoch 6/25
375/375 [==============================] - 9s 25ms/step - loss: 0.3399 - val_loss: 0.4719
Epoch 7/25
375/375 [==============================] - 9s 24ms/step - loss: 0.3194 - val_loss: 0.4495
Epoch 8/25
375/375 [==============================] - 9s 25ms/step - loss: 0.3032 - val_loss: 0.4375
Epoch 9/25
375/375 [==============================] - 9s 24ms/step - loss: 0.2898 - val_loss: 0.4275
Epoch 10/25
375/375 [==============================] - 9s 24ms/step - loss: 0.2789 - val_l

예측
   - 예측도 seq2seq와 동일하나, 추가된 모델 구조를 반영해주어야 함(attention layer)
   - encoder와 decoder를 분리해주었기 때문에 decoder에서 encoder의 은닉상태(estate_h)와 최종 은닉 상태(encoder_outputs)를 따로 입력받아야 함
   - 나머지는 seq2seq에서 작성한 부분과 동일함

In [8]:
encoder_model = Model(inputs=encoder_inputs, 
                      outputs=[encoder_outputs, encoder_states])

In [9]:
decoder_state_input_h = Input(shape=(256))
decoder_state_input_c = Input(shape=(256))

estate_h = Input(shape=(256))
encoder_outputs - Input(shape=(256))

decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)

decoder_states = [state_h, state_c]

S_ = tf.concat([estate_h[:, tf.newaxis, :], decoder_outputs[:, :-1, :]], axis=1)
context_vector = attention([S_, encoder_outputs])
decoder_concat = tf.concat([decoder_outputs, context_vector], axis=1)

decoder_outputs = decoder_softmax_layer(decoder_concat)
decoder_model = Model(inputs=[decoder_inputs, estate_h, encoder_outputs] + decoder_states_inputs,
                      outputs=[decoder_outputs] + decoder_states)

In [10]:
idx_to_src = dict((i, char) for char, i in src_to_idx.items())
idx_to_tar = dict((i, char) for char, i in tar_to_idx.items())

In [11]:
def predict_decode(input_seq):
    outputs_input, states_value = encoder_model.predict(input_seq)
    
    target_seq = np.zeros((1, 1, tar_vocab_size))
    target_seq[0, 0, tar_to_idx['\t']] = 1
    
    stop = False
    decoded_sentence = ""
    
    while not stop:
         output_tokens, h, c = decoder_model.predict([target_seq, states_value[0], outputs_input] + states_value)
         
         sampled_token_index = np.argmax(output_tokens[0, -1, :])
         sampled_char = idx_to_tar[sampled_token_index]
         decoded_sentence += sampled_char
         
         if sampled_char == '\n' or len(decoded_sentence) > max_tar_len:
             stop = True
             
         target_seq = np.zeros((1, 1, tar_vocab_size))
         target_seq[0, 0, sampled_token_index] = 1
         
         states_value = [h, c]

    return decoded_sentence

In [12]:
for seq_index in [0, 1, 2, 3]:
    input_seq = encoder_input[seq_index : seq_index+1]
    decoded_sentence = predict_decode(input_seq)
    
    print("입력 : ", lines.src[seq_index])
    print("정답 : ", lines.tar[seq_index][1:len(lines.tar[seq_index])-1])
    print("번역 : ", decoded_sentence[:len(decoded_sentence)-1], '\n')

ValueError: in user code:

    File "C:\ProgramData\Anaconda3\envs\py37\lib\site-packages\keras\engine\training.py", line 1621, in predict_function  *
        return step_function(self, iterator)
    File "C:\ProgramData\Anaconda3\envs\py37\lib\site-packages\keras\engine\training.py", line 1611, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\ProgramData\Anaconda3\envs\py37\lib\site-packages\keras\engine\training.py", line 1604, in run_step  **
        outputs = model.predict_step(data)
    File "C:\ProgramData\Anaconda3\envs\py37\lib\site-packages\keras\engine\training.py", line 1572, in predict_step
        return self(x, training=False)
    File "C:\ProgramData\Anaconda3\envs\py37\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\ProgramData\Anaconda3\envs\py37\lib\site-packages\keras\engine\input_spec.py", line 248, in assert_input_